In [1]:
import pandas as pd
import numpy as np
import datetime   # datetime 내 timedelta는 몇주전후 까지만 나와 있음
from dateutil.relativedelta import relativedelta # 몇달전후, 몇년전후를 구하기 위함
from tqdm import tqdm
from scipy import stats

In [2]:
data=pd.read_csv('HPAI_Weather_occur.csv', encoding='utf-8-sig')

In [3]:
data.head()

,주소,축종,사육수,진단일,발생일,지역코드,지역명,시도,시군구,location,date,avg_temp(C),lowest_temp(C),highest_temp(C),daily_rain(mm),maxWsWd(16방위),avg_wind(m/s),avg_r_humidity(%)
0,강원도 원주시 귀래면 주포리,닭-산란계,170000,20210224,20210223,114,원주,강원도,원주시,원주,20210223,1.9,-1.2,6.4,0.0,270.0,1.4,28.0
1,강원도 인제군 기린면 북리,닭-산란계,28500,20170107,20170103,211,인제,강원도,인제군,인제,20170103,1.2,-4.0,6.2,0.0,160.0,2.2,60.1
2,강원도 철원군 갈말읍 강포리,닭-산란계,16000,20161212,20161209,95,철원,강원도,철원군,철원,20161209,-0.1,-6.3,3.7,1.7,270.0,1.6,68.9
3,강원도 철원군 갈말읍 강포리,닭-산란계,30000,20161203,20161130,95,철원,강원도,철원군,철원,20161130,3.2,-4.7,8.9,1.2,230.0,0.9,72.1
4,강원도 춘천시 사북면 지촌리,닭-산란계,48,20080507,20080507,101,춘천,강원도,춘천시,춘천,20080507,13.2,7.3,20.9,17.0,180.0,0.6,66.1


In [4]:
# 필요없는 열 삭제
# 평균온도, 일강수량, 평균풍속, 평균상대습도 data만 확보
data_df=data.drop(['축종', '주소', '진단일', '사육수', '지역명', 'location', 'date', 'lowest_temp(C)',
                  'highest_temp(C)', 'maxWsWd(16방위)'], axis=1) 

In [5]:
data_df['발생일']=data_df['발생일'].astype('str')

In [6]:
data_df.head()

,발생일,지역코드,시도,시군구,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,20210223,114,강원도,원주시,1.9,0.0,1.4,28.0
1,20170103,211,강원도,인제군,1.2,0.0,2.2,60.1
2,20161209,95,강원도,철원군,-0.1,1.7,1.6,68.9
3,20161130,95,강원도,철원군,3.2,1.2,0.9,72.1
4,20080507,101,강원도,춘천시,13.2,17.0,0.6,66.1


In [7]:
# 문자열을 datetime 형식으로 바꾸기
data_df['발생일']=data_df['발생일'].map(lambda x : datetime.datetime.strptime(x, '%Y%m%d'))

In [8]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   발생일                1122 non-null   datetime64[ns]
 1   지역코드               1122 non-null   int64         
 2   시도                 1122 non-null   object        
 3   시군구                1122 non-null   object        
 4   avg_temp(C)        1122 non-null   float64       
 5   daily_rain(mm)     1122 non-null   float64       
 6   avg_wind(m/s)      1122 non-null   float64       
 7   avg_r_humidity(%)  1122 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(2)
memory usage: 70.2+ KB


In [9]:
# 월별 데이터 만들기
data_df['month']=data_df['발생일'].map(lambda x: x.month)
data_df['year']=data_df['발생일'].map(lambda x: x.year)

In [10]:
data_df.head()

,발생일,지역코드,시도,시군구,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),month,year
0,2021-02-23,114,강원도,원주시,1.9,0.0,1.4,28.0,2,2021
1,2017-01-03,211,강원도,인제군,1.2,0.0,2.2,60.1,1,2017
2,2016-12-09,95,강원도,철원군,-0.1,1.7,1.6,68.9,12,2016
3,2016-11-30,95,강원도,철원군,3.2,1.2,0.9,72.1,11,2016
4,2008-05-07,101,강원도,춘천시,13.2,17.0,0.6,66.1,5,2008


#### 월별 시군구 데이터와 발생한 날과 발생하지 않은 날의 관계 파악
    1. 시간은 2003~2021년까지 기간
    2. 발생한 날 시군구 별로 월별 기후 평균
    3. 발생하지 않은 날짜를 제외 후 해당하는 달의 시군구 별로 기후 평균
    4. 비교하기

In [11]:
import weatherAPI 

In [12]:
# 발생한 날 시군구 월별 기후 평균 구하기
sigun_df=data_df.groupby(['시도','시군구','month']).mean()

In [13]:
sigun_df.drop(['year'], axis=1, inplace=True)
sigun_df.drop(['지역코드'], axis=1, inplace=True)

In [14]:
sigun_df.reset_index(inplace=True)

In [15]:
for i in range(3,7):
    sigun_df.iloc[:,i]=np.round(sigun_df.iloc[:,i],1)

In [16]:
sigun_df.head()

,시도,시군구,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
0,강원도,원주시,2,1.9,0.0,1.4,28.0
1,강원도,인제군,1,1.2,0.0,2.2,60.1
2,강원도,철원군,11,3.2,1.2,0.9,72.1
3,강원도,철원군,12,-0.1,1.7,1.6,68.9
4,강원도,춘천시,5,13.2,17.0,0.6,66.1


In [17]:
sigun_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   시도                 278 non-null    object 
 1   시군구                278 non-null    object 
 2   month              278 non-null    int64  
 3   avg_temp(C)        278 non-null    float64
 4   daily_rain(mm)     278 non-null    float64
 5   avg_wind(m/s)      278 non-null    float64
 6   avg_r_humidity(%)  278 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 15.3+ KB


In [18]:
# 예시로 일부 살펴보기
sigun_df.query('시군구=="강진군"')

,시도,시군구,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%)
113,전라남도,강진군,1,-0.5,3.8,3.6,68.3
114,전라남도,강진군,2,3.7,0.0,2.0,64.0
115,전라남도,강진군,3,5.2,0.0,3.9,51.1
116,전라남도,강진군,4,16.4,46.0,3.1,87.9
117,전라남도,강진군,5,12.9,0.4,2.6,66.9
118,전라남도,강진군,9,19.8,0.0,1.6,77.4


In [19]:
# 발생일, 발생하지 않은 일 dataset 만들기
cols_name=['other_avg_temp(C)','other_daily_rain(mm)','other_avg_wind(m/s)','other_avg_r_humidity(%)']
for name in cols_name:
    sigun_df[name]=np.nan

In [20]:
data_df['month']=data_df['month'].astype('str')
data_df['year']=data_df['year'].astype('str')

In [21]:
data_df.head()

,발생일,지역코드,시도,시군구,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),month,year
0,2021-02-23,114,강원도,원주시,1.9,0.0,1.4,28.0,2,2021
1,2017-01-03,211,강원도,인제군,1.2,0.0,2.2,60.1,1,2017
2,2016-12-09,95,강원도,철원군,-0.1,1.7,1.6,68.9,12,2016
3,2016-11-30,95,강원도,철원군,3.2,1.2,0.9,72.1,11,2016
4,2008-05-07,101,강원도,춘천시,13.2,17.0,0.6,66.1,5,2008


In [25]:
# 년도와 달 기준으로 발생했던 날짜와 발생하지 않았던 날짜 평균으로 비교.  (한달기준으로 하기)
for i in tqdm(range(len(sigun_df))):
    sigun=sigun_df.loc[i,"시군구"]
    month=str(sigun_df.loc[i,"month"])
    temp=data_df.query('시군구==@sigun and month==@month').reset_index(drop=True)
    temp['발생일']=temp['발생일'].map(lambda x:x.strftime('%Y%m%d'))
    
    temp_total=pd.DataFrame()
    for n, group in temp.groupby(['year']):
        group.reset_index(inplace=True, drop=True)
        if len(group.loc[0,'month'])==1:
            temp_month='0'+group.loc[0,'month']
        else:
            temp_month=group.loc[0,'month']
        start_date=group.loc[0,'year']+temp_month+'01'
        temp_date=datetime.datetime.strptime(start_date, '%Y%m%d')
        end_date=temp_date+relativedelta(months=1)-datetime.timedelta(days=1)
        end_date=end_date.strftime('%Y%m%d')
        temp_df=weatherAPI.load2(start_date, end_date, group.loc[0,'지역코드'])
        for date in group['발생일']:
            for j in range(len(temp_df)):
                if temp_df.loc[j,'date']==date:
                    temp_df=temp_df.drop([j], axis=0)
                    break
            temp_df.reset_index(inplace=True, drop=True)
        temp_total=pd.concat([temp_total,temp_df], axis=0)
        
    for col in cols_name:
        sigun_df.loc[i,col]=np.round(temp_total[col[6:]].mean(),1)

100%|████████████████████████████████████████████████████████████████████████████████| 278/278 [03:46<00:00,  1.22it/s]


In [26]:
sigun_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   시도                       278 non-null    object 
 1   시군구                      278 non-null    object 
 2   month                    278 non-null    int64  
 3   avg_temp(C)              278 non-null    float64
 4   daily_rain(mm)           278 non-null    float64
 5   avg_wind(m/s)            278 non-null    float64
 6   avg_r_humidity(%)        278 non-null    float64
 7   other_avg_temp(C)        278 non-null    float64
 8   other_daily_rain(mm)     278 non-null    float64
 9   other_avg_wind(m/s)      278 non-null    float64
 10  other_avg_r_humidity(%)  278 non-null    float64
dtypes: float64(8), int64(1), object(2)
memory usage: 24.0+ KB


In [30]:
sigun_df.head()

,시도,시군구,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),other_avg_temp(C),other_daily_rain(mm),other_avg_wind(m/s),other_avg_r_humidity(%)
0,강원도,원주시,2,1.9,0.0,1.4,28.0,2.5,0.2,1.2,52.1
1,강원도,인제군,1,1.2,0.0,2.2,60.1,-3.5,0.4,1.7,58.7
2,강원도,철원군,11,3.2,1.2,0.9,72.1,4.3,0.5,1.2,69.8
3,강원도,철원군,12,-0.1,1.7,1.6,68.9,-1.4,2.3,1.0,72.5
4,강원도,춘천시,5,13.2,17.0,0.6,66.1,16.9,3.0,1.2,66.3


In [31]:
sigun_df.to_csv("Weather_month(HPAI_occur_non).csv", index=False, encoding='utf-8-sig')

#### 시군구와 시도의 기후 상관관계 살피기
    1. 시도별로 나눈 후 해당하는 월과 시군구를 두고 관계성을 파악하기
    2. 4가지 기후(온도, 강수량, 풍속, 습도)에 대해 각각 시도와 시군구의 특정월을 기준으로 검증을 해본다
    

In [22]:
sigun_df=pd.read_csv('Weather_month(HPAI_occur_non).csv', encoding='utf-8-sig')

In [23]:
sigun_df['diff_temp']=sigun_df['other_avg_temp(C)']-sigun_df['avg_temp(C)']

In [24]:
diff_temp=sigun_df[['diff_temp']]

In [25]:
minus=diff_temp.query('diff_temp<0')
plus=diff_temp.query('diff_temp>=0')

In [26]:
minus.describe()

,diff_temp
count,127.000000
mean,-2.177953
std,1.741643
min,-7.300000
25%,-3.100000
50%,-1.700000
75%,-0.700000
max,-0.100000


In [27]:
plus.describe()

,diff_temp
count,151.000000
mean,2.691391
std,2.144728
min,0.000000
25%,1.000000
50%,2.200000
75%,4.000000
max,10.000000


In [60]:
sido={}
for n, group in sigun_df.groupby(['시도']):
    group.reset_index(inplace=True, drop=True)
    sido[n]=group

In [67]:
minus_gyeonggi=sido['경기도'].query('diff_temp<0')
plus_gyeonggi=sido['경기도'].query('diff_temp>=0')

In [71]:
set(minus_gyeonggi['시군구'])

{'고양시',
 '광주시',
 '수원시',
 '안성시',
 '양주시',
 '양평군',
 '여주시',
 '연천군',
 '용인시',
 '이천시',
 '파주시',
 '평택시',
 '포천시',
 '화성시'}

In [73]:
minus_gyeonggi.query('시군구=="고양시"')

,시도,시군구,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),other_avg_temp(C),other_daily_rain(mm),other_avg_wind(m/s),other_avg_r_humidity(%),diff_temp
1,경기도,고양시,12,2.3,0.0,0.5,85.5,-3.5,0.2,1.1,68.7,-5.8


In [75]:
data_df.query('시군구=="고양시"') # 한달치는 너무 많을 것..

,발생일,지역코드,시도,시군구,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),month,year
8,2017-03-03,99,경기도,고양시,1.2,0.0,1.8,68.6,3,2017
9,2020-12-28,99,경기도,고양시,2.3,0.0,0.5,85.5,12,2020


In [72]:
set(plus_gyeonggi['시군구'])

{'고양시',
 '과천시',
 '광주시',
 '김포시',
 '남양주시',
 '동두천시',
 '성남시',
 '안성시',
 '양주시',
 '연천군',
 '용인시',
 '의정부시',
 '이천시',
 '파주시',
 '평택시',
 '포천시',
 '화성시'}

In [74]:
plus_gyeonggi.query('시군구=="고양시"')

,시도,시군구,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),other_avg_temp(C),other_daily_rain(mm),other_avg_wind(m/s),other_avg_r_humidity(%),diff_temp
0,경기도,고양시,3,1.2,0.0,1.8,68.6,3.9,0.3,1.4,58.1,2.7


In [69]:
plus_gyeonggi.describe()

,month,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),other_avg_temp(C),other_daily_rain(mm),other_avg_wind(m/s),other_avg_r_humidity(%),diff_temp
count,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000,29.000000
mean,5.551724,0.772414,1.151724,1.962069,57.324138,3.882759,1.065517,1.731034,60.289655,3.110345
std,4.468830,5.975300,3.421217,0.829980,11.234280,6.032949,0.802797,0.580704,5.195351,2.020492
min,1.000000,-9.500000,0.000000,0.800000,29.400000,-5.400000,0.200000,0.900000,48.100000,0.200000
25%,2.000000,-3.200000,0.000000,1.300000,51.500000,-0.600000,0.600000,1.400000,56.600000,1.900000
50%,3.000000,0.800000,0.000000,1.800000,57.700000,3.400000,0.700000,1.700000,60.000000,2.700000
75%,11.000000,2.900000,0.500000,2.400000,63.700000,6.500000,1.200000,1.900000,64.600000,4.100000
max,12.000000,19.000000,17.700000,3.900000,80.300000,20.900000,3.200000,3.400000,69.600000,7.700000


##### 발생일 전일과 발생한 날의 기후 차이로 관계성 찾아보기

In [28]:
cols_name2=['diff_avg_temp(C)','diff_daily_rain(mm)','diff_avg_wind(m/s)','diff_avg_r_humidity(%)']

In [29]:
for col in cols_name2:
    data_df[col]=np.nan

In [30]:
data_df=data_df.sort_values(['발생일']).reset_index(drop=True)

In [31]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1122 entries, 0 to 1121
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   발생일                     1122 non-null   datetime64[ns]
 1   지역코드                    1122 non-null   int64         
 2   시도                      1122 non-null   object        
 3   시군구                     1122 non-null   object        
 4   avg_temp(C)             1122 non-null   float64       
 5   daily_rain(mm)          1122 non-null   float64       
 6   avg_wind(m/s)           1122 non-null   float64       
 7   avg_r_humidity(%)       1122 non-null   float64       
 8   month                   1122 non-null   object        
 9   year                    1122 non-null   object        
 10  diff_avg_temp(C)        0 non-null      float64       
 11  diff_daily_rain(mm)     0 non-null      float64       
 12  diff_avg_wind(m/s)      0 non-null      float64 

In [32]:
# 중복 행 제거
dup_data=data_df.drop_duplicates().reset_index(drop=True)

In [33]:
dup_data.head()

,발생일,지역코드,시도,시군구,avg_temp(C),daily_rain(mm),avg_wind(m/s),avg_r_humidity(%),month,year,diff_avg_temp(C),diff_daily_rain(mm),diff_avg_wind(m/s),diff_avg_r_humidity(%)
0,2003-12-12,127,충청북도,음성군,-1.2,0.0,1.4,74.5,12,2003,NaN,NaN,NaN,NaN
1,2003-12-17,127,충청북도,음성군,0.1,1.8,3.2,63.9,12,2003,NaN,NaN,NaN,NaN
2,2003-12-19,127,충청북도,음성군,-4.7,0.0,4.3,46.1,12,2003,NaN,NaN,NaN,NaN
3,2003-12-21,138,경상북도,경주시,1.4,0.0,4.2,36.6,12,2003,NaN,NaN,NaN,NaN
4,2003-12-21,156,전라남도,나주시,-0.5,0.0,1.1,68.1,12,2003,NaN,NaN,NaN,NaN


In [34]:
dup_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   발생일                     841 non-null    datetime64[ns]
 1   지역코드                    841 non-null    int64         
 2   시도                      841 non-null    object        
 3   시군구                     841 non-null    object        
 4   avg_temp(C)             841 non-null    float64       
 5   daily_rain(mm)          841 non-null    float64       
 6   avg_wind(m/s)           841 non-null    float64       
 7   avg_r_humidity(%)       841 non-null    float64       
 8   month                   841 non-null    object        
 9   year                    841 non-null    object        
 10  diff_avg_temp(C)        0 non-null      float64       
 11  diff_daily_rain(mm)     0 non-null      float64       
 12  diff_avg_wind(m/s)      0 non-null      float64   

In [58]:
# 년도와 달 기준으로 발생했던 날짜와 발생하지 않았던 날짜 비교(1. 온도) (몇일 전 평균인지 살펴보기 (최대 30일))
#total_dup_data={}
for i in range(29,30):
    for j in tqdm(range(len(dup_data))):
        if j!=0:  # 같은 달, 같은 지역일 경우 제일 처음 발생한 기후차이로 데이터 입력
            if (dup_data.loc[j-1,'month']==dup_data.loc[j,'month']
            and dup_data.loc[j-1,'year']==dup_data.loc[j,'year'] and dup_data.loc[j-1,'지역코드']==dup_data.loc[j,'지역코드']):
                for col in cols_name2:
                    dup_data.loc[j,col]=dup_data.loc[j-1,col[5:]]
                continue
        start_date=dup_data.loc[j,'발생일']-datetime.timedelta(days=i)
        start_date=start_date.strftime('%Y%m%d')
        end_date=dup_data.loc[j,'발생일']-datetime.timedelta(days=1)
        end_date=end_date.strftime('%Y%m%d')
        temp_df=weatherAPI.load2(start_date, end_date, dup_data.loc[j,'지역코드'])
        for col in cols_name2:
            dup_data.loc[j,col]=dup_data.loc[j,col[5:]]-np.round(temp_df[col[5:]].mean(),1)
            
    total_dup_data[i]=dup_data.copy()
    positive_temp=dup_data[dup_data[cols_name2[0]]>=0].reset_index(drop=True) #온도만 col[0]
    negative_temp=dup_data[dup_data[cols_name2[0]]<0].reset_index(drop=True)
    m=np.round(len(negative_temp)/len(dup_data),3) 
    if m>0.9:
        print(f'complete index={i}')
        print(f"index={i}, positive={len(positive_temp)}, negative={len(nagative_temp)}, rate={m}")
        break
    else:
        print(f"Fail.. index={i}, positive={len(positive_temp)}, negative={len(negative_temp)}, rate={m}")
        
# 일간 API 초과로 할수 없음

  0%|                                                                                          | 0/841 [00:00<?, ?it/s]

API error


TypeError: 'NoneType' object is not subscriptable

In [69]:
# 저장하기
n=1
for group in total_dup_data.values():
    group.to_pickle('weather_diff_dataset/data_diff_'+str(n)+'.pkl')
    n+=1

In [70]:
# 불러오기
total_dup_df={}
for i in range(1,29):
    total_dup_df[i]=pd.read_pickle('weather_diff_dataset/data_diff_'+str(i)+'.pkl')

In [79]:
cols_name2=['diff_avg_temp(C)','diff_daily_rain(mm)','diff_avg_wind(m/s)','diff_avg_r_humidity(%)']
#온도 살펴보기 col[0]
i=1
for group in total_dup_data.values():
    positive_temp=group[group[cols_name2[0]]>=0].reset_index(drop=True) 
    negative_temp=group[group[cols_name2[0]]<0].reset_index(drop=True)
    m=np.round(len(negative_temp)/len(group),3) 
    if m>0.9:
        print(f'complete index={i}')
        print(f"index={i}, positive={len(positive_temp)}, negative={len(nagative_temp)}, rate={m}")
        break
    else:
        print(f"Fail.. index={i}, positive={len(positive_temp)}, negative={len(negative_temp)}, rate={m}")
    i+=1

Fail.. index=1, positive=476, negative=365, rate=0.434
Fail.. index=2, positive=469, negative=372, rate=0.442
Fail.. index=3, positive=462, negative=379, rate=0.451
Fail.. index=4, positive=461, negative=380, rate=0.452
Fail.. index=5, positive=451, negative=390, rate=0.464
Fail.. index=6, positive=454, negative=387, rate=0.46
Fail.. index=7, positive=449, negative=392, rate=0.466
Fail.. index=8, positive=452, negative=389, rate=0.463
Fail.. index=9, positive=457, negative=384, rate=0.457
Fail.. index=10, positive=469, negative=372, rate=0.442
Fail.. index=11, positive=468, negative=373, rate=0.444
Fail.. index=12, positive=473, negative=368, rate=0.438
Fail.. index=13, positive=469, negative=372, rate=0.442
Fail.. index=14, positive=475, negative=366, rate=0.435
Fail.. index=15, positive=470, negative=371, rate=0.441
Fail.. index=16, positive=469, negative=372, rate=0.442
Fail.. index=17, positive=468, negative=373, rate=0.444
Fail.. index=18, positive=466, negative=375, rate=0.446
Fa

In [80]:
cols_name2=['diff_avg_temp(C)','diff_daily_rain(mm)','diff_avg_wind(m/s)','diff_avg_r_humidity(%)']
#일강수량 살펴보기 col[1]
i=1
for group in total_dup_data.values():
    positive_temp=group[group[cols_name2[1]]>=0].reset_index(drop=True) 
    negative_temp=group[group[cols_name2[1]]<0].reset_index(drop=True)
    m=np.round(len(negative_temp)/len(group),3) 
    if m>0.9:
        print(f'complete index={i}')
        print(f"index={i}, positive={len(positive_temp)}, negative={len(nagative_temp)}, rate={m}")
        break
    else:
        print(f"Fail.. index={i}, positive={len(positive_temp)}, negative={len(negative_temp)}, rate={m}")
    i+=1

Fail.. index=1, positive=672, negative=169, rate=0.201
Fail.. index=2, positive=596, negative=245, rate=0.291
Fail.. index=3, positive=533, negative=308, rate=0.366
Fail.. index=4, positive=484, negative=357, rate=0.424
Fail.. index=5, positive=439, negative=402, rate=0.478
Fail.. index=6, positive=409, negative=432, rate=0.514
Fail.. index=7, positive=377, negative=464, rate=0.552
Fail.. index=8, positive=369, negative=472, rate=0.561
Fail.. index=9, positive=352, negative=489, rate=0.581
Fail.. index=10, positive=336, negative=505, rate=0.6
Fail.. index=11, positive=325, negative=516, rate=0.614
Fail.. index=12, positive=312, negative=529, rate=0.629
Fail.. index=13, positive=305, negative=536, rate=0.637
Fail.. index=14, positive=307, negative=534, rate=0.635
Fail.. index=15, positive=300, negative=541, rate=0.643
Fail.. index=16, positive=289, negative=552, rate=0.656
Fail.. index=17, positive=285, negative=556, rate=0.661
Fail.. index=18, positive=278, negative=563, rate=0.669
Fai

In [81]:
cols_name2=['diff_avg_temp(C)','diff_daily_rain(mm)','diff_avg_wind(m/s)','diff_avg_r_humidity(%)']
#풍속 살펴보기 col[2]
i=1
for group in total_dup_data.values():
    positive_temp=group[group[cols_name2[2]]>=0].reset_index(drop=True) 
    negative_temp=group[group[cols_name2[2]]<0].reset_index(drop=True)
    m=np.round(len(negative_temp)/len(group),3) 
    if m>0.9:
        print(f'complete index={i}')
        print(f"index={i}, positive={len(positive_temp)}, negative={len(nagative_temp)}, rate={m}")
        break
    else:
        print(f"Fail.. index={i}, positive={len(positive_temp)}, negative={len(negative_temp)}, rate={m}")
    i+=1

Fail.. index=1, positive=485, negative=356, rate=0.423
Fail.. index=2, positive=477, negative=364, rate=0.433
Fail.. index=3, positive=478, negative=363, rate=0.432
Fail.. index=4, positive=467, negative=374, rate=0.445
Fail.. index=5, positive=463, negative=378, rate=0.449
Fail.. index=6, positive=461, negative=380, rate=0.452
Fail.. index=7, positive=451, negative=390, rate=0.464
Fail.. index=8, positive=447, negative=394, rate=0.468
Fail.. index=9, positive=446, negative=395, rate=0.47
Fail.. index=10, positive=445, negative=396, rate=0.471
Fail.. index=11, positive=447, negative=394, rate=0.468
Fail.. index=12, positive=447, negative=394, rate=0.468
Fail.. index=13, positive=448, negative=393, rate=0.467
Fail.. index=14, positive=456, negative=385, rate=0.458
Fail.. index=15, positive=452, negative=389, rate=0.463
Fail.. index=16, positive=451, negative=390, rate=0.464
Fail.. index=17, positive=443, negative=398, rate=0.473
Fail.. index=18, positive=448, negative=393, rate=0.467
Fa

In [78]:
cols_name2=['diff_avg_temp(C)','diff_daily_rain(mm)','diff_avg_wind(m/s)','diff_avg_r_humidity(%)']
#상대습도 살펴보기 col[3]
i=1
for group in total_dup_data.values():
    positive_temp=group[group[cols_name2[3]]>=0].reset_index(drop=True) 
    negative_temp=group[group[cols_name2[3]]<0].reset_index(drop=True)
    m=np.round(len(negative_temp)/len(group),3) 
    if m>0.9:
        print(f'complete index={i}')
        print(f"index={i}, positive={len(positive_temp)}, negative={len(nagative_temp)}, rate={m}")
        break
    else:
        print(f"Fail.. index={i}, positive={len(positive_temp)}, negative={len(negative_temp)}, rate={m}")
    i+=1

Fail.. index=1, positive=487, negative=354, rate=0.421
Fail.. index=2, positive=467, negative=374, rate=0.445
Fail.. index=3, positive=484, negative=357, rate=0.424
Fail.. index=4, positive=469, negative=372, rate=0.442
Fail.. index=5, positive=479, negative=362, rate=0.43
Fail.. index=6, positive=478, negative=363, rate=0.432
Fail.. index=7, positive=487, negative=354, rate=0.421
Fail.. index=8, positive=492, negative=349, rate=0.415
Fail.. index=9, positive=495, negative=346, rate=0.411
Fail.. index=10, positive=503, negative=338, rate=0.402
Fail.. index=11, positive=503, negative=338, rate=0.402
Fail.. index=12, positive=498, negative=343, rate=0.408
Fail.. index=13, positive=491, negative=350, rate=0.416
Fail.. index=14, positive=488, negative=353, rate=0.42
Fail.. index=15, positive=483, negative=358, rate=0.426
Fail.. index=16, positive=481, negative=360, rate=0.428
Fail.. index=17, positive=482, negative=359, rate=0.427
Fail.. index=18, positive=486, negative=355, rate=0.422
Fai